In [2]:
import os
import numpy as np
import mdtraj as md
import nglview as nv

In [3]:
state = md.load("c5.pdb")
print(state.topology)
nv.show_mdtraj(state)

<mdtraj.Topology with 1 chains, 3 residues, 22 atoms, 33 bonds>


NGLWidget()

In [12]:
state_tbg = md.load("c5-bond-removed.pdb")
print(state_tbg.topology)
nv.show_mdtraj(state_tbg)

<mdtraj.Topology with 1 chains, 3 residues, 22 atoms, 21 bonds>


NGLWidget()

In [13]:
from bgmol.datasets import AImplicitUnconstrained
dataset = AImplicitUnconstrained(root=os.getcwd()+"/../../", read=True, download=False)

Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.

****** PyMBAR will use 64-bit JAX! *******
* JAX is currently set to 32-bit bitsize *
* which is its default.                  *
*                                        *
* PyMBAR requires 64-bit mode and WILL   *
* enable JAX's 64-bit mode when called.  *
*                                        *
* This MAY cause problems with other     *
* Uses of JAX in the same code.          *
******************************************



Using downloaded and verified file: /tmp/A.pdb


In [14]:
ala_traj = md.Trajectory(dataset.xyz, dataset.system.mdtraj_topology)
print(ala_traj)
nv.show_mdtraj(ala_traj)

<mdtraj.Trajectory with 1000000 frames, 22 atoms, 3 residues, without unitcells>


NGLWidget(max_frame=999999)

In [17]:
ala_traj[0].save_pdb("dataset-sample.pdb")

In [25]:
ala_sample = md.load("dataset-sample.pdb")
print(ala_sample.topology)
nv.show_mdtraj(ala_sample)

<mdtraj.Topology with 1 chains, 3 residues, 22 atoms, 21 bonds>


NGLWidget()

In [44]:
ala_sample.save("test.h5")

In [45]:
loaded = md.load("test.h5")
print(loaded.topology)

<mdtraj.Topology with 1 chains, 3 residues, 22 atoms, 21 bonds>


In [21]:
topology = dataset.system.mdtraj_topology
print(topology)

<Topology; 1 chains, 3 residues, 22 atoms, 21 bonds>
